In [1]:
import os
import sys
import json
import pyspark
import time
import datetime
import configparser

from pymongo import MongoClient
from datetime import date, datetime, timedelta, timezone

from pyspark import StorageLevel
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import *
from pyspark.sql.types import StringType, TimestampType, LongType
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.context import SparkContext, SparkConf

In [2]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--num-executors 96 --total-executor-cores 96 --driver-memory 128g --executor-memory 4g pyspark-shell"

driver_memory = '2g'
executor_memory = '1g'
pyspark_submit_args = ' --driver-memory=' + driver_memory + ' --executor-memory=' + executor_memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

sc = SparkContext()
sc.setLogLevel("OFF")

spark = SparkSession(sc).builder \
    .master('local[*]') \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .appName("attrubution semi") \
    .getOrCreate()

22/04/19 21:37:45 WARN Utils: Your hostname, yangws.local resolves to a loopback address: 127.0.0.1; using 192.168.0.8 instead (on interface en9)
22/04/19 21:37:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/19 21:37:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
tmp_df = spark.read.option("multiline", "true").json("./test.json")

In [25]:
tmp_schema = StructType([
    StructField('action type', StringType(), True),
    StructField('engine', StringType(), True)
])

# OR

tmp_schema = (StructType()
              .add('action type', StringType(), True)
              .add('engine', StringType(), True))

In [26]:
schema_df = spark.read.schema(tmp_schema).option('multiline', 'true').json('./test.json')

In [27]:
schema_df.show()

+-----------+------+
|action type|engine|
+-----------+------+
|       노출|moloco|
+-----------+------+



In [28]:
copy_df = schema_df.select(col('action type').alias('action_type'), col('engine')).alias('copy_df')

In [29]:
copy_df.show()

+-----------+------+
|action_type|engine|
+-----------+------+
|       노출|moloco|
+-----------+------+

